# Lab 5 - Transportation Simplex Method 

<b>Information on group members:</b><br>
1) 156066, Julia Lorenz <br>


In [3]:
import numpy as np
from collections import Counter
import pulp  
import copy
from collections import deque


## Generating data

In [4]:
# generating data
def get_uniform_random_values(n: int, scaling_factor: int):

    # generating n rnadom values by partitioning the interval [0, 1] into n subintervals
    breakpoints = np.sort(np.random.uniform(0, 1, n-1))
    breakpoints = np.concatenate(([0], breakpoints, [1]))

    values = np.diff(breakpoints) * scaling_factor
    values = np.round(values).astype(int)

    difference = scaling_factor - values.sum()

    while difference != 0:
        # avoiding negative values
        if difference > 0:
            # Increase values
            indices = np.where(values > 0)[0]
            idx = np.random.choice(indices)
            values[idx] += 1
            difference -= 1
        elif difference < 0:
            # Decrease values: Avoid reducing to negatives
            indices = np.where(values > 1)[0]
            idx = np.random.choice(indices)
            values[idx] -= 1
            difference += 1 

    return values


def get_cost_matrix(n: int, m: int):

    inf_prob = 0.2
    zero_prob = 0.2

    matrix = np.random.randint(1, 20, size=(n, m)).astype(float)

    if np.random.uniform(0, 1) < inf_prob:
        num_inf = np.random.randint(1, n + 1) 
        inf_indices = np.random.choice(n * m, num_inf, replace=False)
        matrix.flat[inf_indices] = np.inf

    
    if np.random.uniform(0, 1) < zero_prob:
        num_zero = np.random.randint(1, n + 1)
        zero_indices = np.random.choice(n * m, num_zero, replace=False)
        matrix.flat[zero_indices] = 0

    return matrix


def generate_data(problem_size_supply: int, problem_size_demand: int, total_supply_demand: int):

    supply = get_uniform_random_values(problem_size_supply, total_supply_demand)
    demand = get_uniform_random_values(problem_size_demand, total_supply_demand)
    cost = get_cost_matrix(problem_size_supply, problem_size_demand)

    return supply, demand, cost


## Northwest Corner Rule - get initial solution

In [5]:
def northwest_corner_method(supply, demand, cost):

    n = len(supply)
    m = len(demand)

    allocation = np.zeros((n, m), dtype=int)
    i, j = 0, 0
    basic_vars_coords = []
    Z = 0

    while i < n and j < m:
        alloc = min(supply[i], demand[j])
        allocation[i][j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        basic_vars_coords.append((i, j))

        Z += alloc * cost[i][j]
        
        if supply[i] == 0:
            i += 1
        else:
            j += 1


    return allocation, basic_vars_coords, Z

## Computing u, v vectors

In [170]:
def get_u_v_coefs(supply, demand, cost, solution, basic_vars_coords):

    n = len(supply)
    m = len(demand)
    basic_rows = set([coord[0] for coord in basic_vars_coords])
    basic_cols = set([coord[1] for coord in basic_vars_coords])

    u, v = np.full(n, None), np.full(m, None)

    # select a ui value and fix it to 0, the one having the most basic variables with coordinate i
    i_coords = [coord[0] for coord in basic_vars_coords]
    counter = Counter(i_coords)
    most_common_i, _ = counter.most_common(1)[0]
    u[most_common_i] = 0

    queue = deque([("u", most_common_i)])
    u_solved = set([most_common_i])
    v_solved = set()

    while queue:
        var, idx = queue.popleft()
        if var == "u":
            for j in basic_rows:
                if (idx, j) in basic_vars_coords and j not in v_solved:
                    v[j] = cost[idx][j] - u[idx]
                    v_solved.add(j)
                    queue.append(("v", j))
        else:
            for i in basic_cols:
                if (i, idx) in basic_vars_coords and i not in u_solved:
                    u[i] = cost[i][idx] - v[idx]
                    u_solved.add(i)
                    queue.append(("u", i))

    with np.errstate(invalid="ignore"):       
        non_basic_mask = solution == 0
        coefficients = cost - u[:, None] - v  
        
        coefficients[~non_basic_mask] = solution[~non_basic_mask]


    return u, v, coefficients


In [ ]:
def check_optimality(coefficients):
    return np.all(coefficients >= 0)


## Chain rule

In [8]:
def chain_rule(coefficients, solution, basic_vars_coords, cost):

    entering_var = np.unravel_index(np.argmin(coefficients), coefficients.shape)
    entering_var = (entering_var[0], entering_var[1])
    basic_vars_coords.append(entering_var)

    # find path from entering variable back to itself
    path = [entering_var]
    visited = set()
    stack = [(entering_var, True)]  

    while stack:
        current, is_recipient = stack.pop()
        i, j = current

        if len(path) > 1 and current == entering_var:
            break

        if is_recipient:  # Move along column
            candidates = [(x, j) for x, y in basic_vars_coords if y == j and (x, y) not in visited]
        else:  # Move along row
            candidates = [(i, y) for x, y in basic_vars_coords if x == i and (x, y) not in visited]

        found_next = False
        for candidate in candidates:
            stack.append((candidate, not is_recipient)) 
            visited.add(candidate)
            path.append(candidate)
            found_next = True
            break

        if not found_next:
            path.pop()
            if path:
                previous = path[-1]
                stack.append((previous, not is_recipient))

    
    path.pop() # remove the entering variable duplicate from the path
    
    # donors and recipients
    donors = path[1::2]
    recipients = path[0::2]

    min_donor_value = min(solution[cell] for cell in donors)

    leaving_var = None
    for idx, cell in enumerate(path):
        if idx % 2 == 0:  
            solution[cell] += min_donor_value
        else:  
            solution[cell] -= min_donor_value
            if  leaving_var is None and solution[cell] == 0:
                leaving_var = cell

    basic_vars_coords.remove(leaving_var)


    Z = np.sum(solution[solution != 0] * cost[solution != 0])
    
    return solution, basic_vars_coords, Z


## Solving

In [9]:
def solve_transportation_pulp(s, d, cost):
    n_supply = len(s)
    n_demand = len(d)

    prob = pulp.LpProblem("Transportation_Problem", pulp.LpMinimize)
    x = [[pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat="Continuous") for j in range(n_demand)] for i in range(n_supply)]
    prob += pulp.lpSum(cost[i, j] * x[i][j] for i in range(n_supply) for j in range(n_demand) if cost[i, j] != np.inf)

    for i in range(n_supply):
        prob += pulp.lpSum(x[i][j] for j in range(n_demand) if cost[i, j] != np.inf) <= s[i], f"Supply_Constraint_{i}"

    for j in range(n_demand):
        prob += pulp.lpSum(x[i][j] for i in range(n_supply) if cost[i, j] != np.inf) == d[j], f"Demand_Constraint_{j}"

    prob.solve()

    if pulp.LpStatus[prob.status] == "Optimal":
        solution = np.zeros((n_supply, n_demand))
        for i in range(n_supply):
            for j in range(n_demand):
                if cost[i, j] != np.inf:
                    solution[i, j] = pulp.value(x[i][j])
        return solution, pulp.value(prob.objective)
    else:
        print("No optimal solution found.")
        return None, None


In [1]:
def solve_transportation_simplex(supply, demand, cost):
    
    s = copy.deepcopy(supply)
    d = copy.deepcopy(demand)
    c = copy.deepcopy(cost)

    # Pulp implementation
    pulp_solution, pulp_Z = solve_transportation_pulp(s, d, c)
    print("Pulp solution:")
    print(pulp_solution)
    print("Pulp Z:", pulp_Z)
    

    # My implementation (transportation simplex algorithm)
    solution, basic_vars_coords, Z = northwest_corner_method(supply, demand, cost)
    while True:
        u, v, coefficients = get_u_v_coefs(supply, demand, cost, solution, basic_vars_coords)
        if check_optimality(coefficients):
            break
        solution, basic_vars_coords, Z = chain_rule(coefficients, solution, basic_vars_coords, cost)

    print("My implementation solution:")
    print(solution)
    print("My implementation Z:", Z)

    return 

In [ ]:
# toy example from the lab
s = np.array([4, 7, 24, 17, 23, 26])
d = np.array([22, 42, 11, 5, 1, 20])
cost = np.array([[3, 9, 4, 10, 0, 4], [7, 1, 4, 2, 10, 0], [6, 6, 5, 0, 10, 3], [7, 2, 2, 6, 10, 8], [7, 10, 2, 5, 8, 10], [3, 8, 9, 4, 9, 9]])

solve_transportation_simplex(s, d, cost)

Pulp solution:
[[ 0. 30.  0.  0.]
 [10. 10.  0.  0.]
 [30.  0.  0. 50.]
 [ 0.  0. 20. 60.]]
Pulp Z: 370.0
My implementation solution:
[[ 0 30  0  0]
 [20  0  0  0]
 [20 10  0 50]
 [ 0  0 20 60]]
My implementation Z: 370.0


In [224]:
s, d, c = generate_data(6, 6, 400)
solve_transportation_simplex(s, d, c)

Pulp solution:
[[  0.   0.   0.   0.   0.   0.]
 [  0.  63.   0.   0.   0.   0.]
 [ 33.   7.   0.   0.   0.   0.]
 [  6.   0.   0.  97.  24.   0.]
 [136.   0.   0.   0.   0.   1.]
 [  0.  16.  17.   0.   0.   0.]]
Pulp Z: 2418.0
My implementation solution:
[[  0   0   0   0   0   0]
 [  0  63   0   0   0   0]
 [ 33   7   0   0   0   0]
 [  6   0   0  97  24   0]
 [136   0   0   0   0   1]
 [  0  16  17   0   0   0]]
My implementation Z: 2418.0


## Testing

In [61]:
def compare_and_test(supply, demand, cost):
    
    s = copy.deepcopy(supply)
    d = copy.deepcopy(demand)
    c = copy.deepcopy(cost)

    # Pulp implementation
    pulp_solution, pulp_Z = solve_transportation_pulp(s, d, c)

    # My implementation (transportation simplex algorithm)
    iteration = 0
    solution, basic_vars_coords, Z = northwest_corner_method(supply, demand, cost)
    while True:
        u, v, coefficients = get_u_v_coefs(supply, demand, cost, solution, basic_vars_coords)
        if check_optimality(coefficients):
            break
        solution, basic_vars_coords, Z = chain_rule(coefficients, solution, basic_vars_coords, cost)
        iteration += 1

    is_equal_Z = pulp_Z == Z

    return iteration, is_equal_Z

In [225]:
problem_sizes = [5, 10, 15, 20, 25]

for size in problem_sizes:
    iterations = []
    is_equal_Zs = []
    for _ in range(10): 
        s, d, c = generate_data(size, size, 1000)
        iteration, is_equal_Z = compare_and_test(s, d, c)
        iterations.append(iteration)
        is_equal_Zs.append(is_equal_Z)
    print(f"Problem size: {size}, Average iterations: {np.mean(iterations)}, Equal Zs: {np.mean(is_equal_Zs)}")

Problem size: 5, Average iterations: 5.4, Equal Zs: 1.0
Problem size: 10, Average iterations: 18.2, Equal Zs: 1.0
Problem size: 15, Average iterations: 37.5, Equal Zs: 1.0
Problem size: 20, Average iterations: 56.0, Equal Zs: 1.0
Problem size: 25, Average iterations: 85.5, Equal Zs: 1.0
